In [3]:
CSV_PATH = 'youtube.csv'
NUM_HANDS = 2

In [2]:
# Extract landmarks from zip file.
# from dynamic_signs.landmark_extractor import DynamicLandmarkExtractor

if False:
    DLE = DynamicLandmarkExtractor(out_path=CSV_PATH, num_hands=NUM_HANDS)
    DLE.process_video_dataset()
    mp = DLE.mediapiper

In [3]:
# Use this to load landmarks n shit

# Load DATA
from pathlib import Path
from dynamic_signs.youtube_scraper import YouTubeScraper

if False:
    yt = YouTubeScraper(Path("../backend/dynamic_signs/common_words.csv"),Path("../backend/dynamic_signs/video_ids.txt"))
    # yt.find_common_words(min_occurances=50, max=0)
    yt.get_video_signs(max=100,seconds_per_clip=1,num_hands=2)



In [1]:
from dataclasses import dataclass
from dynamic_signs.csv_reader import csv_reader
from sign.CONST import AMOUNT_OF_KEYFRAMES
from sign.trajectory import TrajectoryBuilder, trajectory
CSV_PATH = 'youtube.csv'


reader = csv_reader()
landmarks = reader.extract_two_handed_landmarks(CSV_PATH)
bob = TrajectoryBuilder()
left_hand_trajectories:dict[str,list[trajectory]] = {}
right_hand_trajectories:dict[str,list[trajectory]] = {}

@dataclass
class landmark:
    x:float
    y:float
    z:float

for label,videos in landmarks.items():
    if left_hand_trajectories.get(label) is None:
        left_hand_trajectories[label] = []
    if right_hand_trajectories.get(label) is None:
        right_hand_trajectories[label] = []
    for id,video in videos.items():
        left_frames:list[list[float]] = [] # a list of frames i.e 1 video
        right_frames:list[list[float]] = [] # a list of frames i.e 1 video
        for frame in video:
            left_hand_landmarks,right_hand_landmarks = frame.get_landmarks() # for hands this will be 21 * 3
            
            if left_hand_landmarks is not None:
                left_frames.append(left_hand_landmarks)
            if right_hand_landmarks is not None:
                right_frames.append(right_hand_landmarks)
        
        
        try:
            #TODO: Now we have to extract keyframes, take into account that landmarks are flattened.
            # take the video which has most frames with landmarks and append it. 
            # TODO: This essentially makes it so there can only be one hand pr. video. (not good if we have a sign that requires two hands...)
            if len(left_frames) > len(right_frames):
                if len(left_frames) > AMOUNT_OF_KEYFRAMES: # TODO: This might be to strict

                    trajectory_left = bob.landmark_list_to_trajectories(left_frames)
                    left_hand_trajectories[label].append(bob.landmark_list_to_trajectories(left_frames))
            else:
                if len(right_frames) > AMOUNT_OF_KEYFRAMES: # TODO: This might be to strict
                    trajectory_right = bob.landmark_list_to_trajectories(right_frames)
                    right_hand_trajectories[label].append(bob.landmark_list_to_trajectories(right_frames))

        except Exception as e:
            print("parsed empty list",e)

# for labels,videos in landmarks.items():
#     print(labels,len(videos))







🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
parsed empty list cannot reshape array of size 1 into shape (21,3)
parsed empty list cannot reshape array of size 1 into shape (21,3)
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list cannot reshape array of size 1 into shape (21,3)
parsed empty list cannot reshape array of size 1 into shape (21,3)
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list cannot reshape array of size 1 into shape (21,3)
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by zero
parsed empty list integer modulo by 

In [20]:
from sklearn.tree import DecisionTreeClassifier
labels,flat_trajectory = [],[]

for label, video_trajectories in left_hand_trajectories.items():
    for video in video_trajectories:
        flat_trajectory.append(video.to_numpy_array())
        labels.append(label)
        
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(flat_trajectory,labels)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (60,) + inhomogeneous part.

In [ ]:
# Hvis gini scoren er for høj, kan det så betale sig at fjerne de cases hvor den går i stykker eller vil det bare få modellen til at overfitte?

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


voting_clf = VotingClassifier( estimators=[ ('lr', LogisticRegression(random_state=42)), ('rf', RandomForestClassifier(random_state=42)), ('svc', SVC(random_state=42)) ] ) 

voting_clf.fit(X_train, y_train)